# Regression Analysis


> "_Nature has established patterns originating in the return of events, but only for the most part. New illnesses flood the human race, so that no matter how many experiments you have done on corpses, you have not thereby imposed a limit on the nature of events so that in the future they could not vary._"  - **Gottfried Wilhelm Leibniz**


In this notebook we will discuss several types of regression and how they can be used to solve specific problems. There are a couple caveats associated with regression as well as some common biases. In our discussion, we will only explore a handful of these caveats and biases.

Before we begin our discussion on regresssion, we will take a detour into various estimation approaches. In fact, estimation theory is a branch of statistics that is concerned with estimating the values of numerical characteristics (that is, parameters) based on measured empirical data that has a random component. We will consider three common approaches, namely:
 
 - **Ordinary Least Squares (OLS)**
 - **Maximum Likelihood Estimation (MLE)**
 - **Bayesian (Univariate & Multivariate)**
 
and we'll make efforts to describe them in detail.

We will explore several types of regression namely:

 1. **Linear Regression**
 2. **Ridge Regression**
 3. **Lasso Regression**
 4. **Bayesian Linear Regression**
 5. **Logistic Regression**
 
**NB:** This series is a summary of **Part II: Early Computer-Age Methods** found in **CASI: Computer Age Statistical Inference**.


***


## [Ordinary Least Squares (OLS)](https://en.wikipedia.org/wiki/Ordinary_least_squares)

Ordinary Least Squares (OLS) is a statistical method for approximating the ___unknown parameters___ in a linear regression model by selecting ___parameters___ of linear function from a set of ___explanatory variables___ by the principle of ___least squares___:

> *... minimizing the sum of the squares in the differences between the observed ___dependent variable___ (values of the variable being predicted) in the given dataset and those predicted by the linear function ...*

OLS is consistent if ___regressors___ are exogenous (i.e., independent of the error term) in the linear model, errors are ___homoscedastic___ (have the same finite variance a.k.a., homogeneity of the variance) and are not correlated. This provides us with the ___minimum-variance mean unbiased___ estimation when the errors have finite variances.

> *In other words, OLS is consistent if the independent variables are not dependent on the error term. In the same vein, the errors making up this error term should not be correlated but should have the same finite variance. This will give use the __minimum-variance mean unbiased__ estimation.*

If we add the assumption that the errors are normally distributed (i.e., follow a [Gaussian distribution](https://en.wikipedia.org/wiki/Normal_distribution)), OLS is the ___maximum likelihood estimator___. So, adding the assumption that the errors are Normally distributed to the requirements of OLS consistency makes OLS the ___maximum likelihood estimator___ for the data.

### The linear formulation:

Suppose our data has $n$ observations $\{y_i, x_i\}^n_{i=1}$, where each observation $i$ includes a scalar response $y_i$ and a column vector $x_j$ of values of $p$ predictors (regressors) $x_{ij}$ for $j = 1, ... , p$. In a linear regression model, the response variable, $y_i$, is a linear function of the regressors:

> $y_i = \beta_1x_{i1} + \beta_2x_{i2} + ... + \beta_px_{ip} + \epsilon_i,$

> $y_i = \bf{X}\beta + \epsilon$

A measure of the overall model fit is given by the ___Residual Sum of Squares (RSS)___:

> $S(b) = \sum^{n}_{i=1}(y_i - x_i^Tb)^2 = (y - Xb)^T(y - Xb),$ where T is the traspose.

After estimating $\beta$, the ___fitted values___ (or ___predicted values___) from the regression will be

> $\hat(y) = X\hat{\beta} = Py,$ where $P = X(X^TX)^{-1}X^T$.

It is common to assess the goodness-of-fit of the OLS regression by comparing how much the initial variation in the sample can be reduced by regressing onto $X$. The ___coefficient of determination___ $\bf{R^2}$ is defined as a ratio of "explained" variance to the "total" variance of the dependent variable $y$:

> $R^2 = \frac{\sum(\hat{y_i} - \bar{y})^2}{\sum(y_i - \bar{y})^2} = \frac{y^TP^TLPy}{y^TLy} = 1 - \frac{y^TMy}{y^TLy} = 1 - \frac{RSS}{TSS},$ where TSS is the ___total sum of squares___ for the dependent variable, $L = I_n - \frac{11^T}{n}$.


***


## [Maximum Likelihood Estimation (MLE)](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation)

Maximum likelihood estimation (MLE) is arguably the twentieth century's most influential piece of applied mathematics, and continues to be the method of choice in the statistician's toolkit. Generally, MLE provides nearly unbiased estimates of nearly minimum variance in an automatic way but tends to overfit in regression tasks. So, MLE allows us to obtain estimators -- an estimator attempts to approximate the unknown parameters using the measured data -- by minimising the variance associated with this process of approximation.

**CASI**, _page 91_, discusses the inadequacies and dangers of MLE with reference to the cost of unbiasedness when there are hundreds or thousands of parameters to estimate at the same time. The [___James-Stein Estimator___](https://en.wikipedia.org/wiki/James_stein_estimator) illustrated this point dramatically in 1961 using a few unknown parameters, not hundreds or thousands. Discussing the issues with MLE leads to the story of _shrinkage estimation_ - deliberate introduction of biases to improve overall performance, at a possible danger to individual estimates.


***


## [Bayesian Estimation](https://en.wikipedia.org/wiki/Bayesian_estimation)

Bayesian estimation allows us to perform regression in a manner that can accurately represent our uncertainty (_or certainty_) with an error estimate by optimizing our loss function. Bayesian estimation gives us the predictive distribution:

> Given $D = ((x, y), ... , (x_n, y_n))$ where $x_i, y_i \in \Re$

> The models $Y_1, ..., Y_n$ is independent given $w$ with $Y_i \sim N( w^{t} x_{i}, a^{-1} )$ and $w \sim N(0, b^{-1} I)$

> where $a, b > 0$ assuming $a$ $\&$ $b$ are known.

***

In [1]:
# The usual suspects ...
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# And the accomplices ...
from sklearn import linear_model
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Loading the data
boston = load_boston()
X, y = pd.DataFrame(data=boston.data, columns=boston.feature_names), pd.DataFrame(data=boston.target)

# Train & testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=5)

***

### 1. Linear Regression

#### 1.1. Simple Linear Regression Model

When the data matrix contains only two variables, a constant and a scalar regressor $x_j$, then we have a ___simple regression model___ with parameters $(\alpha, \beta)$:

> $y_i = \alpha + \beta x_i + \epsilon_i.$

The least squares estimates in this case are given by:

> $\hat{\beta} = \frac{\sum x_iy_i - \frac{1}{n}\sum x_i \sum y_i}{\sum x_{i}^{2} - \frac{1}{n}(\sum x_i)^2} = \frac{Cov[x, y]}{Var[x]}$

> $\hat{\alpha} = \bar{y} - \hat{\beta}\bar{x},$

where $Var(.)$ & $Cov(.)$ are sample parameters.

***

We will be using the Boston data set.

#### Boston House Prices dataset
##### Notes
**Data Set Characteristics** 
>- **Number of Instances:** 506
>- **Number of Attributes:** 13
>- Median Value (attribute 14) is usually the target.

|   Attribute   |                               Description                                |
| :-----------: | :----------------------------------------------------------------------: |
|   **CRIM**    |                      per capita crime rate by town                       |
|   **ZN**      |      proportion of residential land zoned for lots over 25,000 sq.ft.    |
|   **INDUS**   |               proportion of non-retail business acres per town           |
|   **CHAS**    |    Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) |
|   **NOX**     |             nitric oxides concentration (parts per 10 million)           |
|   **RM**      |                    average number of rooms per dwelling                  |
|   **AGE**     |            proportion of owner-occupied units built prior to 1940        |
|   **DIS**     |             weighted distances to five Boston employment centres         |
|   **RAD**     |                 index of accessibility to radial highways                |
|   **TAX**     |                 full-value property-tax rate per USD10,000               |
|   **PTRATIO** |                        pupil-teacher ratio by town                       |
|   **B**       |    $1000(B_k - 0.63)^2$ where $B_k$ is the proportion of blacks by town  |
|   **LSTAT**   |                    % lower status of the population                      |
|   **MEDV**    |             Median value of owner-occupied homes in USD1000's            |

>- **Missing Attribute Values:** None
>- **Creator:** Harrison, D. and Rubinfeld, D.L.

In [3]:
# Linear regression model
# Initialization ...
model = linear_model.LinearRegression()

# Fitting the model ...
model.fit(X_train, y_train)

# Printing the coefficients ...
for item in model.coef_:
    for num, coef in enumerate(item):
        print(f'Coefficient {num+1} : {coef}')

Coefficient 1 : -0.1522139594492669
Coefficient 2 : 0.04790174557942721
Coefficient 3 : -0.008088799269282418
Coefficient 4 : 2.58311044816549
Coefficient 5 : -14.69987728585958
Coefficient 6 : 3.9540020585432774
Coefficient 7 : -0.007834301995171976
Coefficient 8 : -1.46170515583877
Coefficient 9 : 0.3452175709811169
Coefficient 10 : -0.012441576683200657
Coefficient 11 : -0.9203519152235027
Coefficient 12 : 0.013362539865919763
Coefficient 13 : -0.518393388938864


From our model we note that we have the same number of coefficients as the number of variables in our data, one coefficient for each variable.

In [3]:
# Prediction & cross validation
y_pred = model.predict(X_test)

# Model evaluation
model.score(X_test, y_test)

0.70562493202054044

***

#### A brief digression on vector norms

##### L2 Norm (_Euclidean Norm_):

> $||\bf{\beta}||_2 = \sqrt{\beta_0^2 +\beta_1^2 + ... + \beta_n^2},$

This is probably the most common method for determining the __norm__ of a vector.


##### L1 Norm:

> $||\bf{\beta}||_1 = |\beta_0| + |\beta_1| + ... + |\beta_n|,$

Not so common, but a more intuitive approach to determining the __norm__ of a vector.

Both these methods have seperate geometric meanings and interpretations, however, __L1__ tends to give values that are larger than __L2__ values. The values we obtain are associated with the interpretations of __Ridge (L2)__ and __Lasso (L1)__. The objective of regression is to obtain ___B.L.U.E. (Best Linear Unbiased Estimators)___ for data that has multiple variables that are highly correlated. That is, the model coefficients will tend to be very high values and dramatically different (_high variance_) each time we sample. This will cause us to question the robustness of our regression model and the reliability of the results that it produces, especially if our model is an ___OLS___. Our goal is to minize these values and take the _best_ from the _worst_.

***

### 2. Ridge Regression (L2)

Linear regression is based on a version of $\hat{\mu}^{MLE}$. Just as in linear regression, we observe _n_-dimensional vectors $y = (y_1, y_2, ..., y_n)^{'}$ from the linear model:

> $y = \bf{X}\beta + \epsilon,$

where $\epsilon$ has uncorrelated components and constant variance $\sigma^2$. The _least squares estimate_ $\hat{\beta}$ is the minimizer of the total sum of squared errors,

> $\hat{\beta} = $ arg min $\{||y - \bf{X}\beta||^{2}_2 + \lambda||\beta||_2\},$ 

> given by $\hat{\beta} = \bf{S}^{-1} \bf{X}^{'} y,$ where $\bf{S}$ is the $p$ x $p$ inner product matrix.

The objective of ridge regression is to minimize the residual error coming from the ___OLS___ estimation subject to some constraints, namely $||\beta|| \leq c$, where $c$ is some constant greater than the __norm (L2)__ of $\beta$.

This is on the backdrop of two motivations:

1. **Too many predictors**: it is not uncommon to have more input variables than observations.
2. **Ill-conditioned X**: computing $\beta_{(least-squares)}$ may be problematic with dramatic flactuations in the values. The ___OLS___ estimator $\beta_{(least-squares)}$ may fit the training data well but will not do so with the test set.

![alt text][ridge]

[ridge]: https://onlinecourses.science.psu.edu/stat857/sites/onlinecourses.science.psu.edu.stat857/files/lesson04/ridge_regression_geomteric/index.png "Geometric Interpretation of Ridge Regression"


Explanation extracted from [here](https://onlinecourses.science.psu.edu/stat857/node/155/):

> The ellipses correspond to the contours of ___residual sum of squares (RSS)___: the inner ellipse has a smaller RSS, and RSS is minimized at ___OLS___. We are trying to minimize the ellipse size and circle simultanously in ridge regression. The ridge estimate is given by the point at which the ellipse and the circle touch.

> There is a trade-off between the penalty term and the RSS. Perhaps a large $\beta$ would give a better RSS but then it will push the penalty term higher. This is why we might actually prefer smaller values of $\beta$ with worse RSS. From an optimization perspective, the penalty term is equivalent to a constraint on the $\beta$'s. The function is still the RSS but now we have a constraint on the $||\beta||$ to be smaller than some constant $c$. There is a correspondence between $\lambda$ and $c$. The larger $\lambda$ is, the more we prefer the $\beta$s to be closer to zero. In the extreme case, when $\lambda = 0$, then we would simply be doing a normal linear regression. On the other hand, as $\lambda$ approaches infinity, all the $\beta$s approach zero.


#### Properties

>- $\hat{\beta}_{least-squares}$ is an unbiased estimator of $\beta$; $\hat{\beta}_{ridge}$ is a biased estimator of $\beta$. The ridge estimator always produces shrinkage towards $0$. $\lambda$ controls the amount of shrinkage. 

>- Another point of note in shrinkage is the ___"effective" degrees of freedom___ associated with a set of parameters. In ridge regression:
 1. If we choose $\lambda = 0$, we have $p$ parameters (since there is no penalty).
 2. If $\lambda$ is large, the parameters are heavily constrained and the degrees of freedom will effectively be lower, approaching zero as $\lambda$ approaches infinity.
 >- The ___effective degrees of freedom___ associated with $\beta_1, \beta_2, ... , \beta_n$ is defined as: $df(\lambda) = tr(\bf{X}(\bf{X}^{'} \bf{X} + \lambda I_p)^{-1} \bf{X}^{'} = \sum^{p}_{j=1} \frac{d^2_j}{d^2_j + \lambda}$
 
 >- There is a $1:1$ mappping between $\lambda$ and the degrees of freedom, son in practice one may simply pick the effective degrees of freedom that one would like  associated with the fit, and solve for $\lambda$.
 
 >- As opposed to arbitrarily choosing $\lambda$, we can make use of _cross-validation_: we select $\lambda$ that yields the smallest cross-validation prediction error.
 
 >- The intercept $\beta_0$ has been left out of the penalty term because $Y$ has been centred. Penalization of the intercept would make the procedure depend on the origin chosen for $Y$.
 
 >- Since the ridge estimator is linear, it is clear how to determine the ___variance-covariance matrix___:
 
 > $var(\hat{\beta}_{ridge}) = \sigma^2(\bf{X}^{'}\bf{X} + \lambda I_p)^{-1} \bf{X}^{'} \bf{X}(\bf{X}^{'} \bf{X} + \lambda I_p)^{-1}$

***

***

### 3. Lasso Regression (L1)

Similar to Ridge regression. We use this type of regression when  we have large number of variables that are also highly correlated. The goal is to minimize the parameters when variables are highly correlated, that is, attempting to minimize the ___sum of squared residuals___ in the ___OLS___ with the contraint that the __norm (L1)__ of $||\beta|| \leq c$, where $c$ is a constant.

> $\hat{\beta} = arg$ min $\{||y - \bf{X}\beta||^{2}_2 + \lambda ||\beta||_1\},$

![alt text][ridge]

[ridge]: https://onlinecourses.science.psu.edu/stat857/sites/onlinecourses.science.psu.edu.stat857/files/lesson05/image_09/index.gif "Geometric Interpretation of Lasso Regression"

Explanation extracted from [here](https://onlinecourses.science.psu.edu/stat857/node/158/):

> Lasso regression performs **L1** shrinkage, so that there are "corners" in the constraint, which in two dimensions corresponds to a diamond. If the ___sum of squares___ "hits" one of these corners, then the coefficient corrending to the axis is shrunk to zero.

> As $p$ increases, the multidimensional diamond has an increasing number of corners, and so it is highly unlikely that some coefficients will be set to zero. Hence, lasso regression performs ___shrinkage___ and (effectively) ___subset selection___.

> In contrast with ___subset selection___, lasso performs a _soft thresholding_: as the smoothing parameter is varied, the sample path of the estimates move continuously to zero.

The motivation for lasso regression stems from the difficulties associated with interpreting the ridge regression solution because it is not sparse (no $\beta$s are set exactly to zero), by asking the question: 

> _"What if we constrain the **L1** norm instead of the **Euclidean (L2) norm**?"_

A subtle but important change $\Rightarrow$ some coefficients will be shrunk to exactly zero.

#### Inferences for Lasso Estimation

The ordinary lasso does not address the uncertainty of parameter estimation; standard errors for $\beta$s are not immediately available. To infer from a lasso estimator, there are several approaches:

> 1. the bootstrap for the estimation of standard errors and deriving an approximate closed form estimate, _Tibshirani (1996)_.

> 2. deriving the sandwich formula in the likelihood setting as an estimator for the covariance of the estimates, _Fan and Li (2001)_.

However, both these approaches approximate covariance matrices given an estimated variance of zero for predictors $\hat{\beta}_j = 0$. The Bayesian lasso (_Park and Casella, (2008)_) provides valid standard errors for $\beta4 and more stable estimates by using the posterior ___median___. The lasso estimate is equivalent to the ___mode___ of the posterior distribution under a normal likelihood and an independent Laplace (double exponential) prior:

> $\pi(\beta) = \frac{\lambda}{2}exp(-\lambda|\beta_j)$

The Bayesian lasso estimates (posterior medians) appear to be a compromise between the ordinary lasso and ridge regression. _Park and Cassella (2008)_ showed that the posterior density was unimodal based on a conditional Laplace prior, $\lambda | \sigma$, a non-informative marginal prior, $\pi(\sigma)^2 \propto \frac{1}{\sigma^2}$, and the availability of a Gibbs algorithm for sampling the posterior distribution.

> $\pi(\beta | \sigma^2) = \prod^p_{j=1} \frac{\lambda}{2\sqrt{\sigma^2}} exp(\frac{\lambda | \beta_j}{2\sqrt{\sigma^2}})$

#### Comparing Ridge and Lasso Regression

- Ridge regression ___shrinks___ all regression coefficients towards zero; lasso tends to give a set of zero regression coefficients and leads to a sparse solution.

- Both ridge and lasso regression coefficients can move from positive to negative values as they are shrunk toward zero.

***

#### A digression on the assumptions of a regression model

An assumption of the fitted model is that the ___standard deviations___ of the error terms are constant and do not depend on the x-value (the predictor) - this is homoscedasticity. It is not required for the estimates to be unbiased, consistent and asymptotically normal.

***

### 4. Bayesian Linear Regression

***


***

### 5. Logistic Regression

***